In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(),'../..')))
import scipy
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn

from Train_fun import train_fun
from Rmse_loss import rmse_loss
from Random_choice import random_choice
from All import all0
from Symmetry import sparse_reg1
from Symmetry import extension
from Select import select

torch.set_default_dtype(torch.float32)
device = torch.device('cuda:0')
np.set_printoptions(suppress=True)
from warnings import filterwarnings
filterwarnings('ignore')

In [8]:
class FiveNet_sin(nn.Module):
    def __init__(self, ni):
        super().__init__()
        self.linear1 = nn.Linear(ni, 128)
        self.linear2 = nn.Linear(128,128)
        self.linear3 = nn.Linear(128, 64)
        self.linear4 = nn.Linear(64,64)
        self.linear5 = nn.Linear(64,1)

    def forward(self, x):
        x = torch.sin(self.linear1(x))
        x = torch.sin(self.linear2(x))
        x = torch.sin(self.linear3(x))
        x = torch.sin(self.linear4(x))
        x = self.linear5(x)
        return x

In [9]:
def diff_RD(net,cor_train):
    cache = np.zeros((1,7))
    n_iter = len(cor_train)//10000
    for i in range(n_iter+1):
        pts = cor_train[i*10000:(i+1)*10000].clone().detach().to(device)
        pts.requires_grad_(True)
        outs = net(pts)
        grad = torch.autograd.grad(outs, pts, grad_outputs=torch.ones_like(outs), create_graph=True)[0]
        ut_t = grad[:,0].reshape(-1,1) 
        ux_t = grad[:,1].reshape(-1,1)
        uy_t = grad[:,2].reshape(-1,1)
        uxx_t= (torch.autograd.grad(grad[:,1].reshape(-1,1), pts, 
                grad_outputs=torch.ones_like(outs), create_graph=True)[0])[:,1].reshape(-1,1)
        uxy_t= (torch.autograd.grad(grad[:,1].reshape(-1,1), pts, 
                grad_outputs=torch.ones_like(outs), create_graph=True)[0])[:,2].reshape(-1,1)
        uyy_t= (torch.autograd.grad(grad[:,2].reshape(-1,1), pts, 
                grad_outputs=torch.ones_like(outs), create_graph=True)[0])[:,2].reshape(-1,1)
        uyy = uyy_t.cpu().detach().numpy()    
        uxx = uxx_t.cpu().detach().numpy()
        uxy = uxy_t.cpu().detach().numpy()
        ux = ux_t.cpu().detach().numpy()
        uy = uy_t.cpu().detach().numpy()
        ut = ut_t.cpu().detach().numpy()
        u_pred = outs.cpu().detach().numpy()
        del ux_t, ut_t, uy_t, grad, uxx_t, uyy_t, uxy_t, outs
        cache = np.vstack((cache,np.hstack((u_pred,ux,uy,uxx,uxy,uyy,ut))))
    return cache[1:]

In [3]:
data = scipy.io.loadmat('reaction_diffusion_standard.mat') # grid 256*256*201

t_steps = np.real(data['t'].flatten()[:,None])
x_steps = np.real(data['x'].flatten()[:,None])
y_steps = np.real(data['y'].flatten()[:,None])
Exact_u = data['u']
Exact_v = data['v']

X, Y, T = np.meshgrid(x_steps, y_steps, t_steps)

X_star = np.hstack((T.flatten()[:,None],X.flatten()[:,None], Y.flatten()[:,None]))
u_star = Exact_u.flatten()[:,None] 
v_star = Exact_v.flatten()[:,None]   

In [4]:
xx, yy, tt = np.meshgrid(x_steps[1:-1], y_steps[1:-1], t_steps[1:-1])
X_star = np.hstack((tt.flatten()[:,None], xx.flatten()[:,None], yy.flatten()[:,None]))
U_star = np.hstack((Exact_u[1:-1,1:-1,1:-1].flatten()[:,None], 
                        Exact_v[1:-1,1:-1,1:-1].flatten()[:,None]))
data = np.hstack((X_star,U_star))

In [5]:
t_cor = np.unique(data[:,0])
x_cor = np.unique(data[:,1])
y_cor = np.unique(data[:,2])
#data = data[np.round(data[:,0]/0.05)%2==0]
#data = data[np.round((data[:,1]-x_cor[0])/0.078125)%3==0]
#data = data[np.round((data[:,2]-y_cor[0])/0.078125)%3==0]

In [6]:
data = data[data[:,0]<5]
data = data[np.abs(data[:,1])<4]
data = data[np.abs(data[:,2])<4]
data = data.astype('float32')
u = data[:,3].reshape(-1,1)
v = data[:,4].reshape(-1,1)

In [10]:
per = 0.2
np.random.seed(0)
cor_train = torch.from_numpy(data[:,:3])

u_clean = data[:,3].reshape(-1,1)
u_noise = u_clean + per*np.std(u_clean)*np.random.randn(u_clean.shape[0],u_clean.shape[1])
u_noise = u_noise.astype('float32')
u_train = torch.from_numpy(u_noise).reshape(-1,1)

v_clean = data[:,4].reshape(-1,1)
v_noise = v_clean + per*np.std(v_clean)*np.random.randn(v_clean.shape[0],v_clean.shape[1])
v_noise = v_noise.astype('float32')
v_train = torch.from_numpy(v_noise).reshape(-1,1)

In [11]:
netu = FiveNet_sin(3).to(device)
try:
    netu.load_state_dict(torch.load('net/netu_{}'.format(per)))
except:
    train_fun(netu,cor_train,u_train,N_red_lr=4,epochs=5000,lr=0.001,threshold=-1)
    torch.save(netu.state_dict(),'net/netu_{}'.format(per))
    
netv = FiveNet_sin(3).to(device)
try:
    netv.load_state_dict(torch.load('net/netv_{}'.format(per)))
except:
    train_fun(netv,cor_train,v_train,N_red_lr=4,epochs=5000,lr=0.001,threshold=-1)
    torch.save(netv.state_dict(),'net/netv_{}'.format(per))

In [12]:
cache = diff_RD(netu,cor_train)
u_pred = cache[:,0].reshape(-1,1)
ux = cache[:,1].reshape(-1,1)
uy = cache[:,2].reshape(-1,1)
uxx = cache[:,3].reshape(-1,1)
uxy = cache[:,4].reshape(-1,1)
uyy = cache[:,5].reshape(-1,1)
ut = cache[:,-1].reshape(-1,1)

cache = diff_RD(netv,cor_train)
v_pred = cache[:,0].reshape(-1,1)
vx = cache[:,1].reshape(-1,1)
vy = cache[:,2].reshape(-1,1)
vxx = cache[:,3].reshape(-1,1)
vxy = cache[:,4].reshape(-1,1)
vyy = cache[:,5].reshape(-1,1)
vt = cache[:,-1].reshape(-1,1)

In [13]:
rmse_loss(0.1*(vxx+vyy)-u*v**2-u**3-v**3-u**2*v+v,vt)

tensor(0.0219, device='cuda:0', dtype=torch.float64)

In [14]:
# make variable institution accroding the u_result_pre
cache = np.hstack((u_pred,v_pred,vx,vy,vxx,vxy,vyy,vt))
cache = random_choice(cache,50000)
# choose new variable
x = extension(cache[:,:-1],deg=3,state=2)
lin_reg = sparse_reg1(cache,deg=3,state=2,tol=0.05,alpha=10**-6)
x_feature = x.T[np.where(np.abs(lin_reg.coef_)>0,True,False)].T
feature = np.hstack((x_feature,cache[:,-1].reshape(-1,1)))

0 tensor(0.0308, device='cuda:0', dtype=torch.float64)
1 tensor(0.0279, device='cuda:0', dtype=torch.float64)
2 tensor(0.0364, device='cuda:0', dtype=torch.float64)
3 tensor(0.0308, device='cuda:0', dtype=torch.float64)
4 tensor(0.0308, device='cuda:0', dtype=torch.float64)
5 tensor(0.0308, device='cuda:0', dtype=torch.float64)
6 tensor(0.0340, device='cuda:0', dtype=torch.float64)


In [15]:
state  = feature.shape[1] - 1
print(state)

7


In [19]:
LC, exp_list, exp = all0('resultv_{}.txt'.format(per),feature,deg=2,state=state,
                         compare_threshold=1,tol=0.05,alpha=10**-5,
                         niterations=300, data_points=1000)

Started!

Expressions evaluated per second: 6.930e+05
Head worker occupation: 9.5%
Progress: 1592 / 4500 total iterations (35.378%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           3.840e-01  1.594e+01  y = 0.06318
3           3.712e-02  1.168e+00  y = (x₁ * -1.2352)
5           1.694e-03  1.543e+00  y = ((x₁ + x₃) * -1.0456)
7           1.693e-03  3.751e-04  y = (((x₁ + x₃) - 0.0010844) * -1.0454)
8           1.280e-03  2.796e-01  y = sin((x₁ * -1.2696) - x₃)
10          1.279e-03  2.704e-04  y = sin((x₁ / -0.78767) - (x₃ / 0.99773))
12          1.089e-03  8.075e-02  y = (sin((x₁ * -1.2696) - x₃) - (-0.020751 * x₆))
14          1.075e-03  6.468e-03  y = sin((x₁ * (((x₆ * -0.17747) * x₆) - 1.243)) - x₃)
16          1.039e-03  1.709e-02  y = sin((x₁ * ((((x₆ + x₂) * -0.17747) * x₆) - 1.243)) - x₃)
18          1.023e-03  7.437e-03  y = sin((x₁ * ((((x₆ + x₂) * -0.17747) 

0 tensor(0.0279, device='cuda:0', dtype=torch.float64)
1 tensor(0.0745, device='cuda:0', dtype=torch.float64)
2 tensor(0.6184, device='cuda:0', dtype=torch.float64)
3 tensor(0.2947, device='cuda:0', dtype=torch.float64)
4 tensor(0.0745, device='cuda:0', dtype=torch.float64)
5 tensor(0.0618, device='cuda:0', dtype=torch.float64)
[ 0.          0.88005241 -1.00473917 -0.99424607 -0.87457313  0.09321116
  0.09217813  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.        ]
[[0.         1.         1.05426404 0.06974393 0.67274362 0.67456555]
 [0.         2.         0.81024577 0.0493651  0.38249246 0.38341091]
 [0.         3.         1.08909406 0.00340098 0.88719082 0.05436003]
 [0.         4.         0.50442669 0.62393796 0.51418348 0.43963931]
 [0.         5.         0.50517981 0.62338783 0.51320019 0.4424099 ]
 [1.   

1 tensor(0.0745, device='cuda:0', dtype=torch.float64)
2 tensor(0.8032, device='cuda:0', dtype=torch.float64)
3 tensor(0.0745, device='cuda:0', dtype=torch.float64)
[ 0.          0.87973417 -1.00214592 -0.87428411  0.09266329  0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.        ]
[[0.         1.         1.16691208 0.07590481 0.6681618  0.67241947]
 [0.         2.         1.08871713 0.00338285 0.88688301 0.05432563]
 [0.         3.         0.66930734 0.82690488 0.55129734 0.40818626]
 [1.         2.         0.07436762 1.09137845 0.6257203  0.62241755]
 [1.         3.         0.88822257 0.73786647 0.45099512 0.50717942]
 [2.         3.         0.78508834 0.63461715 0.34429112 0.48581117]]
[[0.         1.         0.07590481 0.11360244 1.        ]
 [0.         2.         0.00338285 0.06226983 1.        ]
 [0.         3.         0.40818626 0.74041035 3.        ]
 [1.         2.         0.07436762 0.11948187 0.        ]
 [1.         3.     

Started!

Expressions evaluated per second: 7.310e+05
Head worker occupation: 7.1%
Progress: 1618 / 4500 total iterations (35.956%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           3.790e-01  1.594e+01  y = 0.056298
5           3.789e-01  3.015e-05  y = ((-8.2857e-07 / x₀) + 0.05657)
7           3.770e-01  2.570e-03  y = ((x₀ * (x₀ * -0.2409)) - -0.093584)
9           3.767e-01  3.559e-04  y = ((0.32658 * ((-0.093549 - x₀) * x₀)) - -0.093549)
10          3.749e-01  4.784e-03  y = cube(((-0.3653 - x₀) * x₀) - -0.45423)
11          3.716e-01  8.906e-03  y = (sin(cube(x₀ - 0.93969)) * -0.17127)
13          3.712e-01  4.752e-04  y = (sin(cube(x₀ - 0.93969) - -0.083951) * -0.16522)
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 7.460e+0

In [21]:
k = select(LC,0.05)
exp_list[k]

[1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1]
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]


-x0 + 0.09732849*x1